In [1]:
import os

import torch
import wandb
from hydra import compose, initialize
from torch.utils.data import DataLoader

from src.utils.load import load_dvinp
from src.utils.visualize import visualize_dvinp_both

In [2]:
try:
    import torch_directml
    device = torch_directml.device() 
except ImportError:
    device = torch.device("cpu") 

In [3]:
dir = "../models/test_dis_informed" 

In [4]:
with initialize(version_base=None, config_path=dir):
    cfg = compose(config_name="cfg")

    dvinp, trainer = load_dvinp(cfg, device)

    dvinp_path = f"{dir}/dvinp.pth"
    optim_path = f"{dir}/optim.pth"

    if os.path.exists(dvinp_path):
        dvinp_state_dict = torch.load(dvinp_path, map_location=torch.device("cpu"))
        dvinp.load_state_dict(dvinp_state_dict)
        print(f"loaded dvinp from {dvinp_path}")

    if os.path.exists(optim_path):
        optim_state_dict = torch.load(optim_path, map_location=torch.device('cpu'))
        trainer.optimizer.load_state_dict(optim_state_dict)
        print(f"loaded optim from {optim_path}")

Generating tasks: 100%|██████████| 4096/4096 [00:00<00:00, 295252.71it/s]
Dropped Escape call with ulEscapeCode : 0x03007703


In [5]:
wandb_logging = False

trainer.wandb_logging = wandb_logging

if wandb_logging:
    wandb.init(project="test-new")

In [6]:
# trainer.optimizer.param_groups[0]["lr"] = 0.2 * cfg.training.learning_rate
# print(trainer.optimizer.param_groups[0]["lr"])

In [ ]:
num_epochs = 100

losses = trainer.train(
    num_epochs=num_epochs,
    max_clip_norm=cfg.training.max_clip_norm,
    alpha=cfg.training.alpha,
    validate=True
)

In [ ]:
max_context_size = 5
num_samples = 2500 # num_cells will be root
gen_dataloader = DataLoader(trainer.dataset, 1, True)

ranges = [(-5, 5), (-5, 5)]

targets, samples = visualize_dvinp_both(
    device=device,
    dvinp=dvinp,
    dataloader=gen_dataloader,
    num_samples=num_samples,
    max_context_size=max_context_size,
    ranges=ranges,
)

In [21]:
torch.save(dvinp.state_dict(), f"{dir}/dvinp.pth")